In [2]:
!export CUDA_VISIBLE_DEVICES=0

'export' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [4]:
torch.cuda.is_available()

True

In [5]:
torch.cuda.device_count()

1

In [6]:
torch.cuda.current_device()

0

In [7]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [1]:
import torch
torch.cuda.is_available()
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np
import pandas as pd

dfs = []
alldf =pd.DataFrame()

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-bertclass-test.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',0)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', True)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
    force_cudnn_initialization()

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            # from test_bertclass import Solver
            from Whisper_Biclass_bert_test import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            assert paras.load is not None
            from Whisper_Biclass_bert_train import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    for idx in range(5):
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass-test-5-{idx+1}.yaml'
        config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
        config['core']['ckpt'] = f'LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}_sd0/bert-v2-15k-r-a-final-s12.pth'
        solver = Solver(config, paras, mode)

        solver.load_data()
    #     solver.print_model()
        solver.set_model()
        names, hyps, txts, ans = solver.exec()
        dfs.append(pd.DataFrame(
            {'name': names,
             'hyps': hyps,
             'truth': txts,
             'ifcorrect': ans
            }))
        print(dfs[idx])
        print(names)
        print(hyps)
        print(txts)
        print(ans)
        del solver


0


In [2]:
main()


Using test mode

[INFO] Evaluating result of tr. config @ ../LAS_Mandarin_PyTorch-master/config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1.yaml
[INFO] Evaluating result of tr. ckpt @ LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1_sd0/bert-v2-15k-r-a-final-s12.pth


Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-1-16']
Mozillacv11Dataset CTT5-1-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 100
text len: 100
remove None, then wav data: 99, text len: 99
Setup ASR model and optimizer 
('15',)   0.819206714630127   0.0   False
('16',)   0.4743093252182007   1.0   False
('22',)   0.37050679326057434   0.0   True
('28',)   0.5112345814704895   1.0   True
('34',)   0.08283716440200806   0.0   True
('35',)   0.4790976941585541   1.0   False
('40',)   0.9470027089118958   1.0   True
('42',)   0.21387498080730438   0.0   True
('51',)   0.44306302070617676   0.0   True
('53',)   0.07446487247943878   0.0   True
('55',)   0.06

Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-2-16']
Mozillacv11Dataset CTT5-2-16 found wav data: 18
text len: 18
remove None, then wav data: 18, text len: 18
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 100
text len: 100
remove None, then wav data: 99, text len: 99
Setup ASR model and optimizer 
('11',)   0.903518795967102   1.0   True
('25',)   0.19898059964179993   1.0   False
('2',)   0.010419154539704323   0.0   True
('37',)   0.2594561278820038   1.0   False
('39',)   0.997884213924408   1.0   True
('45',)   0.24165275692939758   1.0   False
('46',)   0.970781683921814   1.0   True
('65',)   0.4213246703147888   1.0   False
('67',)   0.6454043984413147   1.0   True
('68',)   0.9548993110656738   0.0   False
('71',)   0.3837

...

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-3-16']
Mozillacv11Dataset CTT5-3-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 100
text len: 100
remove None, then wav data: 99, text len: 99
Setup ASR model and optimizer 
('18',)   0.21773295104503632   1.0   False
('1',)   0.06277923285961151   0.0   True
('20',)   0.38756898045539856   1.0   False
('30',)   0.009022320620715618   0.0   True
('32',)   0.1245959997177124   1.0   False
('38',)   0.49307405948638916   1.0   False
('3',)   0.0309127364307642   0.0   True
('43',)   0.03194970265030861   0.0   True
('52',)   0.2529307007789612   0.0   True
('57',)   0.021242694929242134   1.0   False
('58',)  

Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-4-16']
Mozillacv11Dataset CTT5-4-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 100
text len: 100
remove None, then wav data: 99, text len: 99
Setup ASR model and optimizer 
('100',)   0.028600512072443962   0.0   True
('12',)   0.787111222743988   1.0   True
('17',)   0.8725988864898682   1.0   True
('21',)   0.29837700724601746   0.0   True
('29',)   0.051589906215667725   0.0   True
('4',)   0.7856533527374268   1.0   True
('54',)   0.03437100723385811   0.0   True
('56',)   0.029669761657714844   0.0   True
('63',)   0.285675048828125   1.0   False
('6',)   0.06031308323144913   0.0   True
('7',)   0.020

Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-5-16']
Mozillacv11Dataset CTT5-5-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 100
text len: 100
remove None, then wav data: 99, text len: 99
Setup ASR model and optimizer 
('10',)   0.3048495352268219   1.0   False
('13',)   0.41156837344169617   0.0   True
('14',)   0.011722947470843792   0.0   True
('24',)   0.9183430075645447   1.0   True
('31',)   0.46399256587028503   1.0   False
('44',)   0.048655588179826736   0.0   True
('47',)   0.29054588079452515   0.0   True
('50',)   0.09732697159051895   0.0   True
('5',)   0.4932619333267212   1.0   False
('64',)   0.6036778688430786   1.0   True
('69',)   0

In [3]:
for idx, df in enumerate(dfs):
    alldf = pd.concat([alldf, df])
    # df.to_csv(f'./biclass_result/bert15k-{idx+1}_2', index=False)
alldf.to_csv(f'./biclass_result/bert-v2-15k-r-a-final-s12.csv', index=False)

In [ ]:
print('asd')

In [ ]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv

Traceback (most recent call last):
  File "eval.py", line 33, in <module>
    result['hyp_char_cnt'] = result.apply(lambda x: len(x.hyp), axis=1)
  File "c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\pandas\core\frame.py", line 9423, in apply
    return op.apply().__finalize__(self, method="apply")
  File "c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\pandas\core\apply.py", line 678, in apply
    return self.apply_standard()
  File "c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\pandas\core\apply.py", line 798, in apply_standard
    results, res_index = self.apply_series_generator()
  File "c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\pandas\core\apply.py", line 814, in apply_series_generator
    results[i] = self.f(v)
  File "eval.py", line 33, in <lambda>
    result['hyp_char_cnt'] = result.apply(lambda x: len(x.hyp), axis=1)
  File "c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\pandas\core\generic.py", line 5989, in __getattr__
    return

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
import numpy as np
y_true = [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0] 
y_pred = [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
target_names = ['non-patient', 'patient']
print(classification_report(y_true, y_pred, target_names=target_names))
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print('sensitivity: ',tp / (fn+tp))
print('specificity: ',tn / (fp+tn))
pred = np.array([0.0443, 0.0140, 0.3337, 0.0560, 0.0385, 0.5118, 0.1961, 0.1746, 0.0268, 0.2939, 0.0169, 0.6525, 0.3661, 0.1010, 0.0077, 0.0994, 0.2264, 0.6143, 0.0750, 0.4048, 0.2368])
fpr, tpr, thresholds = metrics.roc_curve(np.array(y_true), np.array(pred), pos_label=1)
print('AUC: ',metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

 non-patient       0.80      0.92      0.86        13
     patient       0.83      0.62      0.71         8

    accuracy                           0.81        21
   macro avg       0.82      0.77      0.79        21
weighted avg       0.81      0.81      0.80        21

sensitivity:  0.625
specificity:  0.9230769230769231
AUC:  0.5288461538461539
